<a href="https://colab.research.google.com/github/Timndg2030/Assignment-1/blob/master/Capstone_Project_Finding_the_Ideal_Restaurant_In_Cenral_London%2C_England.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project - Finding the Ideal Restaurant In Cenral London, England


## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>


The aim of the project is to find the Assist with a business plan for a possible investment in opening a restaurant in Central London, England. The project will aim to find the optimal location.

The project will aim to provide an insight into the most ideal location for a retaurant and the type of restaurant, in order to advice the Investment company to further approach the absolute franchise, which would be perfect for the location. Though Machine Learning we will be able to aggregate, wranglle data, along with the use of Forsquare to arrive at the sort out locations.

In the project, the **Data Section** will be a  description of the data that will be used to solve the problem and the sources. **Methodology** section will discussion and description of exploratory data analysis carried out, any inferential statistical testing performed, and if any machine learnings were used establishing the strategy and purposes. The
**Results** section will discuss the results and the 
Discussion Section will elaborate and discuss any observations noted and any recommendations suggested based on the results. Finally, the **Conclusion** section will report conclusion.








## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to the current restaurants in the neighborhood, if any
* distance of neighborhood from Oxford Circus

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Foursquare**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Central London will be obtained using **Foursquare** of well known Central London location (Oxford Circus)

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas in Cental London that have low restaurant density, particularly those with low number of restaurants. We will limit our analysis to area ~15 miles around city center.

In first step we have collected the required **data: location and type (category) of every restaurant within 6km from Central London** (Oxford Circus). 

Second step in our analysis will be calculation and exploration of '**restaurant density**' across different areas - we will use **heatmaps** to identify a few promising areas close to center with low number of restaurants in general and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than two restaurants in radius of 250 meters**, and we want locations **without a restaurants in radius of 400 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.